<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-packages" data-toc-modified-id="Import-packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import packages</a></span></li><li><span><a href="#Intialize-dataset" data-toc-modified-id="Intialize-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Intialize dataset</a></span></li><li><span><a href="#Compute-min/max" data-toc-modified-id="Compute-min/max-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Compute min/max</a></span></li><li><span><a href="#Create-Cluster" data-toc-modified-id="Create-Cluster-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create Cluster</a></span></li><li><span><a href="#Compute-Histogram-for-CLT-variable" data-toc-modified-id="Compute-Histogram-for-CLT-variable-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Compute Histogram for CLT variable</a></span></li><li><span><a href="#Display-results-with-VCS" data-toc-modified-id="Display-results-with-VCS-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Display results with VCS</a></span></li></ul></div>

# Import packages

Use conda to install distributed and dask in your environmnent. (If you are using Jupyter Notebook or Jupyter Lab, the simplest thing to do is to open a new command line window and activate the same CDAT environment that you are using for this notebook then run the following line of code in the new command line window.)
  * **conda install -c conda-forge distributed dask**

In [2]:
import cdms2
import cdat_info
import dask.array as da
from dask.distributed import LocalCluster
from dask.distributed  import Client
import vcs
import vcsaddons

/Users/davis278/miniconda3/envs/nightly/lib/python3.6/site-packages/unidata/__init__.py:2: UserWarning: unidata package is deprecated please use genutil.udunits instead of unidata.udunits
  warnings.warn("unidata package is deprecated please use genutil.udunits instead of unidata.udunits")


Download some sample data.

In [5]:
vcs.download_sample_data_files()

Downloading: 'BlueMarble.ppm' from 'https://cdat.llnl.gov/cdat/sample_data/' in: /Users/davis278/miniconda3/envs/nightly/share/cdat/sample_data/BlueMarble.ppm
Downloading: 'clt.nc' from 'https://cdat.llnl.gov/cdat/sample_data/' in: /Users/davis278/miniconda3/envs/nightly/share/cdat/sample_data/clt.nc
Downloading: 'geo.1deg.ctl' from 'https://cdat.llnl.gov/cdat/sample_data/' in: /Users/davis278/miniconda3/envs/nightly/share/cdat/sample_data/geo.1deg.ctl
Downloading: 'geo.1deg.gmp' from 'https://cdat.llnl.gov/cdat/sample_data/' in: /Users/davis278/miniconda3/envs/nightly/share/cdat/sample_data/geo.1deg.gmp
Downloading: 'geo.1deg.grb' from 'https://cdat.llnl.gov/cdat/sample_data/' in: /Users/davis278/miniconda3/envs/nightly/share/cdat/sample_data/geo.1deg.grb
Downloading: 'meshfill.nc' from 'https://cdat.llnl.gov/cdat/sample_data/' in: /Users/davis278/miniconda3/envs/nightly/share/cdat/sample_data/meshfill.nc
Downloading: 'navy_land.nc' from 'https://cdat.llnl.gov/cdat/sample_data/' in: /

# Intialize dataset
Convert File Varible into a Dask Array

In [6]:
f = cdms2.open(cdat_info.get_sampledata_path()+"/clt.nc")
dataFV=f["clt"]
myDaskArray= da.from_array(dataFV, chunks=(1,46,72))

In [7]:
myDaskArray

,Array,Chunk
Bytes,1.59 MB,13.25 kB
Shape,"(120, 46, 72)","(1, 46, 72)"
Count,121 Tasks,120 Chunks
Type,float32,cdms2.TransientVariable


# Compute min/max

In [8]:
min = myDaskArray.min()
max = myDaskArray.max()
print(min.compute(), max.compute())

0.0 100.0


# Create Cluster

In [9]:
cluster=LocalCluster(n_workers=5,threads_per_worker=2)
display(cluster)
c=Client(cluster)
c

Client Scheduler: tcp://127.0.0.1:59924 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 10 Memory: 17.18 GB


# Compute Histogram for CLT variable

In [10]:
f = cdms2.open(cdat_info.get_sampledata_path()+"/clt.nc")
dataFV=f["clt"]
myDaskArray= da.from_array(dataFV, chunks=(1,46,72))
h,bins=da.histogram(myDaskArray,bins=100,range=[1,100])
bins
results = h.compute()

# Display results with VCS

In [11]:
plt=vcs.init()
plt.plot(bins,results)